In [1]:
#Conventional One

In [2]:
#--------------------------------#
#         House-keeping          #
#--------------------------------#

import numpy
import math
import time
from scipy.stats import norm
from joblib import Parallel, delayed
import multiprocessing
import sys

In [3]:
#Number of Workers
num_cores = 1

In [4]:
# Grid for x
nx            = 1500;
xmin          = 0.1;
xmax          = 4.0;

In [5]:
# Grid for e: parameters for Tauchen
ne            = 15;
ssigma_eps    = 0.02058;
llambda_eps   = 0.99;
m             = 1.5;

In [6]:
# Utility function
ssigma        = 2;
bbeta         = 0.97;
T             = 10;

In [7]:
# Prices
r             = 0.07;
w             = 5;

In [8]:
# Initialize the grid for X
xgrid = numpy.zeros(nx)

In [9]:
# Initialize the grid for E and the transition probability matrix
egrid = numpy.zeros(ne)
P     = numpy.zeros((ne, ne))

In [10]:
# Initialize value function V
V     = numpy.zeros((T, nx, ne))

In [15]:
print(len(V)) # Number of period
print(len(V[0])) # For each age, possible state values
print(len(V[0][0])) # For each state x, possible shocks

10
1500
15


In [16]:
#--------------------------------#
#         Grid creation          #
#--------------------------------#

# Function to construct the grid for capital (x)
size = nx;
xstep = (xmax - xmin) /(size - 1);
it = 0;
for i in range(0,nx):
    xgrid[i] = xmin + it*xstep;
    it = it+1;

In [17]:
xgrid # Possible state variable values

array([0.1       , 0.10260173, 0.10520347, ..., 3.99479653, 3.99739827,
       4.        ])

In [18]:
# Function to construct the grid for productivity (e) using Tauchen (1986)
size = ne;
ssigma_y = math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2)));
estep = 2*ssigma_y*m / (size-1);
it = 0;
for i in range(0,ne):
    egrid[i] = (-m*math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2))) + it*estep);
    it = it+1;

In [19]:
egrid # possible 15 shocks

array([-0.21883163, -0.18756997, -0.15630831, -0.12504664, -0.09378498,
       -0.06252332, -0.03126166,  0.        ,  0.03126166,  0.06252332,
        0.09378498,  0.12504664,  0.15630831,  0.18756997,  0.21883163])

In [20]:
# Function to construct the transition probability matrix for productivity (P) using Tauchen (1986)
mm = egrid[1] - egrid[0];
for j in range(0,ne):
    for k in range(0,ne):
        if (k == 0):
            P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps);
        elif (k == ne-1):
            P[j, k] = 1 - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);
        else:
            P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps) - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);

In [23]:
# Exponential of the grid e
for i in range(0,ne):
	egrid[i] = math.exp(egrid[i]);

In [24]:
#--------------------------------#
#     Structure and function     #
#--------------------------------#

# Value function
VV = math.pow(-10.0, 5); # just power function -10^5

In [25]:
# Data structure of state and exogenous variables
class modelState(object):
    def __init__(self,ind,ne,nx,T,age,P,xgrid,egrid,ssigma,bbeta,w,r):
        self.ind		= ind # combination of state vars
        self.ne			= ne # number of point on e-grid (shock)
        self.nx			= nx # number of point on x-grid (wealth)
        self.T			= T #last age (death)
        self.age		= age #age
        self.P			= P #transition prob matrix
        self.xgrid		= xgrid #state var - maybe saving?
        self.egrid		= egrid #possible shock grid
        self.ssigma		= ssigma #utility fct CRRA coefficient
        self.bbeta		= bbeta #discount factor
        self.w			= w #wage
        self.r			= r #interest rate 

In [26]:
modelState

__main__.modelState

In [27]:
# Function that returns value for a given state
# ind: a unique state that corresponds to a pair (ie,ix) - ie - labor productivity shock!
def value_func(states): #a fct that receives a 'state object'

	ind = states.ind 
	ne = states.ne
	nx = states.nx
	T = states.T
	age = states.age
	P = states.P
	xgrid = states.xgrid
	egrid = states.egrid
	ssigma = states.ssigma
	bbeta = states.bbeta
	w = states.w
	r = states.r

#maht.floor - largest integer less than or equal to ind/ne
	ix = int(math.floor(ind/ne)); #just dividing
	ie = int(math.floor(ind%ne)); #remainder after ind divided by ne .. 17%5 = 2

	VV = math.pow(-10.0, 3) #-10^3 - bottom of utility I guess..?
	for ixp in range(0,nx): # - for every possible next wealth state,
        #1. Form a expectation about the future
		expected = 0.0; # calculate the expectation
		if(age < T-1): #before you die, 
			for iep in range(0,ne): # - for every possible next realization of the shock 
				expected = expected + P[ie, iep]*V[age+1, ixp, iep] # form an expectation - for last period... haha no error
        #If you are not in the last period, expectation is not zero
        #2. Possible consumption functions
		cons  = (1 + r)*xgrid[ix] + egrid[ie]*w - xgrid[ixp]; #current state interest rate, wage, next period saving
        
        #3. Utility calculation
		utility = math.pow(cons, (1-ssigma))/(1-ssigma) + bbeta*expected;
        
        #Boundary.. / Borrowing limit (incomplete market)
		if(cons <= 0):
			utility = math.pow(-10.0,5); #any big super negative number will work
		
		if(utility >= VV): # 최대값을 걸러내는 장소 .. 어떤 state에서 다음 어떤 state로 갈지 결정하는거지 
            #다음 state를 1로 했을 때 만약에 다음 state를 0이라고 잡은 value function보다 높으면 고 아니면 업데이트 안한다.
            #사실 이 function은 value function만 계산하고 optimal policy는 안보인다?
			VV = utility;

		utility = 0.0;

	return[VV]; #so for this function, for given state, it results in the maximum value that household can achieve!

In [98]:
#--------------------------------#
#     Life-cycle computation     #
#--------------------------------#

print(" ")
print("Life cycle computation: ")
print(" ")


start = time.time()

for age in reversed(range(0,T)): #starting from "T"

	# This function computes `value_func` in parallel for all the states
	results = Parallel(n_jobs=num_cores)(delayed(value_func)(modelState(ind,ne,nx,T,age,P,xgrid,egrid,ssigma,bbeta,w,r)) for ind in range(0,nx*ne))

	# I write the results on the value matrix: V
	for ind in range(0,nx*ne): # for every case for shocks and state vars
		
		ix = int(math.floor(ind/ne)); # current state vars 
		ie = int(math.floor(ind%ne)); # current shocks

		V[age, ix, ie] = results[ind][0];

	finish = time.time() - start
	print("Age: ", age+1, ". Time: ", round(finish, 4), " seconds.")

finish = time.time() - start
print("TOTAL ELAPSED TIME: ", round(finish, 4), " seconds. \n")


#---------------------#
#     Some checks     #
#---------------------#

print(" - - - - - - - - - - - - - - - - - - - - - \n")
print("The first entries of the value function: \n")

for i in range(0,3):
	print(round(V[0, 0, i], 5)) # 소수점 5번째 자리까지만 표시

print(" \n")

 
Life cycle computation: 
 
Age:  10 . Time:  65.1433  seconds.
Age:  9 . Time:  463.7921  seconds.
Age:  8 . Time:  847.9525  seconds.
Age:  7 . Time:  1231.3666  seconds.


KeyboardInterrupt: 

In [37]:
# Faster Version with numba

In [1]:
#--------------------------------#
#         House-keeping          #
#--------------------------------#

from numba import jit, jitclass, njit, prange, int64, float64
import numpy
import math
import time
from scipy.stats import norm
from collections import OrderedDict
import sys


In [2]:
#--------------------------------#
#         Initialization         #
#--------------------------------#

# Number of workers

# Grid for x
nx            = 1500;
xmin          = 0.1;
xmax          = 4.0;

# Grid for e: parameters for Tauchen
ne            = 15;
ssigma_eps    = 0.02058;
llambda_eps   = 0.99;
m             = 1.5;

# Utility function
ssigma        = 2;
bbeta         = 0.97;
T             = 10;

# Prices
r             = 0.07;
w             = 5;

# Initialize the grid for X
xgrid = numpy.zeros(nx)

# Initialize the grid for E and the transition probability matrix
egrid = numpy.zeros(ne)
P     = numpy.zeros((ne, ne))


#--------------------------------#
#         Grid creation          #
#--------------------------------#

# Function to construct the grid for capital (x)
size = nx;
xstep = (xmax - xmin) /(size - 1);
it = 0;
for i in range(0,nx):
	xgrid[i] = xmin + it*xstep;
	it = it+1;


# Function to construct the grid for productivity (e) using Tauchen (1986)
size = ne;
ssigma_y = math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2)));
estep = 2*ssigma_y*m / (size-1);
it = 0;
for i in range(0,ne):
	egrid[i] = (-m*math.sqrt(math.pow(ssigma_eps, 2) / (1 - math.pow(llambda_eps,2))) + it*estep);
	it = it+1;


# Function to construct the transition probability matrix for productivity (P) using Tauchen (1986)
mm = egrid[1] - egrid[0];
for j in range(0,ne):
	for k in range(0,ne):
		if (k == 0):
			P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps);
		elif (k == ne-1):
			P[j, k] = 1 - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);
		else:
			P[j, k] = norm.cdf((egrid[k] - llambda_eps*egrid[j] + (mm/2))/ssigma_eps) - norm.cdf((egrid[k] - llambda_eps*egrid[j] - (mm/2))/ssigma_eps);


# Exponential of the grid e
for i in range(0,ne):
	egrid[i] = math.exp(egrid[i]);



#--------------------------------#
#     Structure and function     #
#--------------------------------#

# Value function
VV = math.pow(-10.0, 5);

specs = OrderedDict()
specs['ind'] = int64
specs['ne'] = int64
specs['nx'] = int64
specs['T'] = int64
specs['age'] = int64
specs['P'] = float64[:,:]
specs['xgrid'] = float64[:]
specs['egrid'] = float64[:]
specs['ssigma'] = float64
specs['bbeta'] = float64
specs['w'] = float64
specs['r'] = float64
specs['V'] = float64[:,:,:]


# Data structure of state and exogenous variables
@jitclass(specs)
class modelState(object):
	def __init__(self,ind,ne,nx,T,age,P,xgrid,egrid,ssigma,bbeta,w,r,V):
		self.ind		= ind
		self.ne			= ne
		self.nx			= nx
		self.T			= T
		self.age		= age
		self.P			= P
		self.xgrid		= xgrid
		self.egrid		= egrid
		self.ssigma		= ssigma
		self.bbeta		= bbeta
		self.w			= w
		self.r			= r
		self.V			= V

# Function that returns value for a given state
# ind: a unique state that corresponds to a pair (ie,ix)
@njit
def value_func(states):

	ind = states.ind
	ne = states.ne
	nx = states.nx
	T = states.T
	age = states.age
	P = states.P
	xgrid = states.xgrid
	egrid = states.egrid
	ssigma = states.ssigma
	bbeta = states.bbeta
	w = states.w
	r = states.r
	V = states.V

	ix = int(math.floor(ind/ne));
	ie = int(math.floor(ind%ne));

	VV = math.pow(-10.0, 3)
	for ixp in range(0,nx):
		expected = 0.0;
		if(age < T-1):
			for iep in range(0,ne):
				expected = expected + P[ie, iep]*V[age+1, ixp, iep]

		cons  = (1 + r)*xgrid[ix] + egrid[ie]*w - xgrid[ixp];

		utility = math.pow(cons, (1-ssigma))/(1-ssigma) + bbeta*expected;

		if(cons <= 0):
			utility = math.pow(-10.0,5);
		
		if(utility >= VV):
			VV = utility;

		utility = 0.0;

	return[VV];



#--------------------------------#
#     Life-cycle computation     #
#--------------------------------#

print(" ")
print("Life cycle computation: ")
print(" ")

@njit(parallel=True)
def compute(age, V):

	for ind in prange(0,nx*ne):

		states = modelState(ind, ne, nx, T, age, P, xgrid, egrid, ssigma, bbeta, w, r, V)
		
		ix = int(math.floor(ind/ne));
		ie = int(math.floor(ind%ne));

		V[age, ix, ie] = value_func(states)[0];

	return(V)



start = time.time()
# Initialize value function V
V     = numpy.zeros((T, nx, ne))

for age in range(T-1, -1, -1):
	V = compute(age, V)

	finish = time.time() - start
	print("Age: ", age+1, ". Time: ", round(finish, 4), " seconds.")


finish = time.time() - start
print("TOTAL ELAPSED TIME: ", round(finish, 4), " seconds. \n")


#---------------------#
#     Some checks     #
#---------------------#

print(" - - - - - - - - - - - - - - - - - - - - - \n")
print("The first entries of the value function: \n")

for i in range(0,3):
	print(round(V[0, 0, i], 5))

print(" \n")

 
Life cycle computation: 
 
Age:  10 . Time:  1.6341  seconds.
Age:  9 . Time:  2.0313  seconds.
Age:  8 . Time:  2.4034  seconds.
Age:  7 . Time:  2.7918  seconds.
Age:  6 . Time:  3.2167  seconds.
Age:  5 . Time:  3.6217  seconds.
Age:  4 . Time:  4.0001  seconds.
Age:  3 . Time:  4.3747  seconds.
Age:  2 . Time:  4.7577  seconds.
Age:  1 . Time:  5.1706  seconds.
TOTAL ELAPSED TIME:  5.1726  seconds. 

 - - - - - - - - - - - - - - - - - - - - - 

The first entries of the value function: 

-2.11762
-2.07729
-2.02366
 



In [8]:
	for ind in prange(0,nx*ne):

		states = modelState(ind, ne, nx, T, age, P, xgrid, egrid, ssigma, bbeta, w, r, V)
		
		ix = int(math.floor(ind/ne));
		ie = int(math.floor(ind%ne));
		print(ix)
		print(ie)

0
0
0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
12
0
13
0
14
1
0
1
1
1
2
1
3
1
4
1
5
1
6
1
7
1
8
1
9
1
10
1
11
1
12
1
13
1
14
2
0
2
1
2
2
2
3
2
4
2
5
2
6
2
7
2
8
2
9
2
10
2
11
2
12
2
13
2
14
3
0
3
1
3
2
3
3
3
4
3
5
3
6
3
7
3
8
3
9
3
10
3
11
3
12
3
13
3
14
4
0
4
1
4
2
4
3
4
4
4
5
4
6
4
7
4
8
4
9
4
10
4
11
4
12
4
13
4
14
5
0
5
1
5
2
5
3
5
4
5
5
5
6
5
7
5
8
5
9
5
10
5
11
5
12
5
13
5
14
6
0
6
1
6
2
6
3
6
4
6
5
6
6
6
7
6
8
6
9
6
10
6
11
6
12
6
13
6
14
7
0
7
1
7
2
7
3
7
4
7
5
7
6
7
7
7
8
7
9
7
10
7
11
7
12
7
13
7
14
8
0
8
1
8
2
8
3
8
4
8
5
8
6
8
7
8
8
8
9
8
10
8
11
8
12
8
13
8
14
9
0
9
1
9
2
9
3
9
4
9
5
9
6
9
7
9
8
9
9
9
10
9
11
9
12
9
13
9
14
10
0
10
1
10
2
10
3
10
4
10
5
10
6
10
7
10
8
10
9
10
10
10
11
10
12
10
13
10
14
11
0
11
1
11
2
11
3
11
4
11
5
11
6
11
7
11
8
11
9
11
10
11
11
11
12
11
13
11
14
12
0
12
1
12
2
12
3
12
4
12
5
12
6
12
7
12
8
12
9
12
10
12
11
12
12
12
13
12
14
13
0
13
1
13
2
13
3
13
4
13
5
13
6
13
7
13
8
13
9
13
10
13
11
13
12
13
13
13
14
14
0
14
1
14
2
14
3
14
4
14
5


6
105
7
105
8
105
9
105
10
105
11
105
12
105
13
105
14
106
0
106
1
106
2
106
3
106
4
106
5
106
6
106
7
106
8
106
9
106
10
106
11
106
12
106
13
106
14
107
0
107
1
107
2
107
3
107
4
107
5
107
6
107
7
107
8
107
9
107
10
107
11
107
12
107
13
107
14
108
0
108
1
108
2
108
3
108
4
108
5
108
6
108
7
108
8
108
9
108
10
108
11
108
12
108
13
108
14
109
0
109
1
109
2
109
3
109
4
109
5
109
6
109
7
109
8
109
9
109
10
109
11
109
12
109
13
109
14
110
0
110
1
110
2
110
3
110
4
110
5
110
6
110
7
110
8
110
9
110
10
110
11
110
12
110
13
110
14
111
0
111
1
111
2
111
3
111
4
111
5
111
6
111
7
111
8
111
9
111
10
111
11
111
12
111
13
111
14
112
0
112
1
112
2
112
3
112
4
112
5
112
6
112
7
112
8
112
9
112
10
112
11
112
12
112
13
112
14
113
0
113
1
113
2
113
3
113
4
113
5
113
6
113
7
113
8
113
9
113
10
113
11
113
12
113
13
113
14
114
0
114
1
114
2
114
3
114
4
114
5
114
6
114
7
114
8
114
9
114
10
114
11
114
12
114
13
114
14
115
0
115
1
115
2
115
3
115
4
115
5
115
6
115
7
115
8
115
9
115
10
115
11
115
12
115
13
11

202
14
203
0
203
1
203
2
203
3
203
4
203
5
203
6
203
7
203
8
203
9
203
10
203
11
203
12
203
13
203
14
204
0
204
1
204
2
204
3
204
4
204
5
204
6
204
7
204
8
204
9
204
10
204
11
204
12
204
13
204
14
205
0
205
1
205
2
205
3
205
4
205
5
205
6
205
7
205
8
205
9
205
10
205
11
205
12
205
13
205
14
206
0
206
1
206
2
206
3
206
4
206
5
206
6
206
7
206
8
206
9
206
10
206
11
206
12
206
13
206
14
207
0
207
1
207
2
207
3
207
4
207
5
207
6
207
7
207
8
207
9
207
10
207
11
207
12
207
13
207
14
208
0
208
1
208
2
208
3
208
4
208
5
208
6
208
7
208
8
208
9
208
10
208
11
208
12
208
13
208
14
209
0
209
1
209
2
209
3
209
4
209
5
209
6
209
7
209
8
209
9
209
10
209
11
209
12
209
13
209
14
210
0
210
1
210
2
210
3
210
4
210
5
210
6
210
7
210
8
210
9
210
10
210
11
210
12
210
13
210
14
211
0
211
1
211
2
211
3
211
4
211
5
211
6
211
7
211
8
211
9
211
10
211
11
211
12
211
13
211
14
212
0
212
1
212
2
212
3
212
4
212
5
212
6
212
7
212
8
212
9
212
10
212
11
212
12
212
13
212
14
213
0
213
1
213
2
213
3
213
4
213
5
213
6
2

6
312
7
312
8
312
9
312
10
312
11
312
12
312
13
312
14
313
0
313
1
313
2
313
3
313
4
313
5
313
6
313
7
313
8
313
9
313
10
313
11
313
12
313
13
313
14
314
0
314
1
314
2
314
3
314
4
314
5
314
6
314
7
314
8
314
9
314
10
314
11
314
12
314
13
314
14
315
0
315
1
315
2
315
3
315
4
315
5
315
6
315
7
315
8
315
9
315
10
315
11
315
12
315
13
315
14
316
0
316
1
316
2
316
3
316
4
316
5
316
6
316
7
316
8
316
9
316
10
316
11
316
12
316
13
316
14
317
0
317
1
317
2
317
3
317
4
317
5
317
6
317
7
317
8
317
9
317
10
317
11
317
12
317
13
317
14
318
0
318
1
318
2
318
3
318
4
318
5
318
6
318
7
318
8
318
9
318
10
318
11
318
12
318
13
318
14
319
0
319
1
319
2
319
3
319
4
319
5
319
6
319
7
319
8
319
9
319
10
319
11
319
12
319
13
319
14
320
0
320
1
320
2
320
3
320
4
320
5
320
6
320
7
320
8
320
9
320
10
320
11
320
12
320
13
320
14
321
0
321
1
321
2
321
3
321
4
321
5
321
6
321
7
321
8
321
9
321
10
321
11
321
12
321
13
321
14
322
0
322
1
322
2
322
3
322
4
322
5
322
6
322
7
322
8
322
9
322
10
322
11
322
12
322
13
32

440
12
440
13
440
14
441
0
441
1
441
2
441
3
441
4
441
5
441
6
441
7
441
8
441
9
441
10
441
11
441
12
441
13
441
14
442
0
442
1
442
2
442
3
442
4
442
5
442
6
442
7
442
8
442
9
442
10
442
11
442
12
442
13
442
14
443
0
443
1
443
2
443
3
443
4
443
5
443
6
443
7
443
8
443
9
443
10
443
11
443
12
443
13
443
14
444
0
444
1
444
2
444
3
444
4
444
5
444
6
444
7
444
8
444
9
444
10
444
11
444
12
444
13
444
14
445
0
445
1
445
2
445
3
445
4
445
5
445
6
445
7
445
8
445
9
445
10
445
11
445
12
445
13
445
14
446
0
446
1
446
2
446
3
446
4
446
5
446
6
446
7
446
8
446
9
446
10
446
11
446
12
446
13
446
14
447
0
447
1
447
2
447
3
447
4
447
5
447
6
447
7
447
8
447
9
447
10
447
11
447
12
447
13
447
14
448
0
448
1
448
2
448
3
448
4
448
5
448
6
448
7
448
8
448
9
448
10
448
11
448
12
448
13
448
14
449
0
449
1
449
2
449
3
449
4
449
5
449
6
449
7
449
8
449
9
449
10
449
11
449
12
449
13
449
14
450
0
450
1
450
2
450
3
450
4
450
5
450
6
450
7
450
8
450
9
450
10
450
11
450
12
450
13
450
14
451
0
451
1
451
2
451
3
451
4

...

1006
5
1006
6
1006
7
1006
8
1006
9
1006
10
1006
11
1006
12
1006
13
1006
14
1007
0
1007
1
1007
2
1007
3
1007
4
1007
5
1007
6
1007
7
1007
8
1007
9
1007
10
1007
11
1007
12
1007
13
1007
14
1008
0
1008
1
1008
2
1008
3
1008
4
1008
5
1008
6
1008
7
1008
8
1008
9
1008
10
1008
11
1008
12
1008
13
1008
14
1009
0
1009
1
1009
2
1009
3
1009
4
1009
5
1009
6
1009
7
1009
8
1009
9
1009
10
1009
11
1009
12
1009
13
1009
14
1010
0
1010
1
1010
2
1010
3
1010
4
1010
5
1010
6
1010
7
1010
8
1010
9
1010
10
1010
11
1010
12
1010
13
1010
14
1011
0
1011
1
1011
2
1011
3
1011
4
1011
5
1011
6
1011
7
1011
8
1011
9
1011
10
1011
11
1011
12
1011
13
1011
14
1012
0
1012
1
1012
2
1012
3
1012
4
1012
5
1012
6
1012
7
1012
8
1012
9
1012
10
1012
11
1012
12
1012
13
1012
14
1013
0
1013
1
1013
2
1013
3
1013
4
1013
5
1013
6
1013
7
1013
8
1013
9
1013
10
1013
11
1013
12
1013
13
1013
14
1014
0
1014
1
1014
2
1014
3
1014
4
1014
5
1014
6
1014
7
1014
8
1014
9
1014
10
1014
11
1014
12
1014
13
1014
14
1015
0
1015
1
1015
2
1015
3
1015
4
1015
5
101

1119
0
1119
1
1119
2
1119
3
1119
4
1119
5
1119
6
1119
7
1119
8
1119
9
1119
10
1119
11
1119
12
1119
13
1119
14
1120
0
1120
1
1120
2
1120
3
1120
4
1120
5
1120
6
1120
7
1120
8
1120
9
1120
10
1120
11
1120
12
1120
13
1120
14
1121
0
1121
1
1121
2
1121
3
1121
4
1121
5
1121
6
1121
7
1121
8
1121
9
1121
10
1121
11
1121
12
1121
13
1121
14
1122
0
1122
1
1122
2
1122
3
1122
4
1122
5
1122
6
1122
7
1122
8
1122
9
1122
10
1122
11
1122
12
1122
13
1122
14
1123
0
1123
1
1123
2
1123
3
1123
4
1123
5
1123
6
1123
7
1123
8
1123
9
1123
10
1123
11
1123
12
1123
13
1123
14
1124
0
1124
1
1124
2
1124
3
1124
4
1124
5
1124
6
1124
7
1124
8
1124
9
1124
10
1124
11
1124
12
1124
13
1124
14
1125
0
1125
1
1125
2
1125
3
1125
4
1125
5
1125
6
1125
7
1125
8
1125
9
1125
10
1125
11
1125
12
1125
13
1125
14
1126
0
1126
1
1126
2
1126
3
1126
4
1126
5
1126
6
1126
7
1126
8
1126
9
1126
10
1126
11
1126
12
1126
13
1126
14
1127
0
1127
1
1127
2
1127
3
1127
4
1127
5
1127
6
1127
7
1127
8
1127
9
1127
10
1127
11
1127
12
1127
13
1127
14
1128
0
112

1227
9
1227
10
1227
11
1227
12
1227
13
1227
14
1228
0
1228
1
1228
2
1228
3
1228
4
1228
5
1228
6
1228
7
1228
8
1228
9
1228
10
1228
11
1228
12
1228
13
1228
14
1229
0
1229
1
1229
2
1229
3
1229
4
1229
5
1229
6
1229
7
1229
8
1229
9
1229
10
1229
11
1229
12
1229
13
1229
14
1230
0
1230
1
1230
2
1230
3
1230
4
1230
5
1230
6
1230
7
1230
8
1230
9
1230
10
1230
11
1230
12
1230
13
1230
14
1231
0
1231
1
1231
2
1231
3
1231
4
1231
5
1231
6
1231
7
1231
8
1231
9
1231
10
1231
11
1231
12
1231
13
1231
14
1232
0
1232
1
1232
2
1232
3
1232
4
1232
5
1232
6
1232
7
1232
8
1232
9
1232
10
1232
11
1232
12
1232
13
1232
14
1233
0
1233
1
1233
2
1233
3
1233
4
1233
5
1233
6
1233
7
1233
8
1233
9
1233
10
1233
11
1233
12
1233
13
1233
14
1234
0
1234
1
1234
2
1234
3
1234
4
1234
5
1234
6
1234
7
1234
8
1234
9
1234
10
1234
11
1234
12
1234
13
1234
14
1235
0
1235
1
1235
2
1235
3
1235
4
1235
5
1235
6
1235
7
1235
8
1235
9
1235
10
1235
11
1235
12
1235
13
1235
14
1236
0
1236
1
1236
2
1236
3
1236
4
1236
5
1236
6
1236
7
1236
8
1236
9
123

1332
4
1332
5
1332
6
1332
7
1332
8
1332
9
1332
10
1332
11
1332
12
1332
13
1332
14
1333
0
1333
1
1333
2
1333
3
1333
4
1333
5
1333
6
1333
7
1333
8
1333
9
1333
10
1333
11
1333
12
1333
13
1333
14
1334
0
1334
1
1334
2
1334
3
1334
4
1334
5
1334
6
1334
7
1334
8
1334
9
1334
10
1334
11
1334
12
1334
13
1334
14
1335
0
1335
1
1335
2
1335
3
1335
4
1335
5
1335
6
1335
7
1335
8
1335
9
1335
10
1335
11
1335
12
1335
13
1335
14
1336
0
1336
1
1336
2
1336
3
1336
4
1336
5
1336
6
1336
7
1336
8
1336
9
1336
10
1336
11
1336
12
1336
13
1336
14
1337
0
1337
1
1337
2
1337
3
1337
4
1337
5
1337
6
1337
7
1337
8
1337
9
1337
10
1337
11
1337
12
1337
13
1337
14
1338
0
1338
1
1338
2
1338
3
1338
4
1338
5
1338
6
1338
7
1338
8
1338
9
1338
10
1338
11
1338
12
1338
13
1338
14
1339
0
1339
1
1339
2
1339
3
1339
4
1339
5
1339
6
1339
7
1339
8
1339
9
1339
10
1339
11
1339
12
1339
13
1339
14
1340
0
1340
1
1340
2
1340
3
1340
4
1340
5
1340
6
1340
7
1340
8
1340
9
1340
10
1340
11
1340
12
1340
13
1340
14
1341
0
1341
1
1341
2
1341
3
1341
4
134

10
1452
11
1452
12
1452
13
1452
14
1453
0
1453
1
1453
2
1453
3
1453
4
1453
5
1453
6
1453
7
1453
8
1453
9
1453
10
1453
11
1453
12
1453
13
1453
14
1454
0
1454
1
1454
2
1454
3
1454
4
1454
5
1454
6
1454
7
1454
8
1454
9
1454
10
1454
11
1454
12
1454
13
1454
14
1455
0
1455
1
1455
2
1455
3
1455
4
1455
5
1455
6
1455
7
1455
8
1455
9
1455
10
1455
11
1455
12
1455
13
1455
14
1456
0
1456
1
1456
2
1456
3
1456
4
1456
5
1456
6
1456
7
1456
8
1456
9
1456
10
1456
11
1456
12
1456
13
1456
14
1457
0
1457
1
1457
2
1457
3
1457
4
1457
5
1457
6
1457
7
1457
8
1457
9
1457
10
1457
11
1457
12
1457
13
1457
14
1458
0
1458
1
1458
2
1458
3
1458
4
1458
5
1458
6
1458
7
1458
8
1458
9
1458
10
1458
11
1458
12
1458
13
1458
14
1459
0
1459
1
1459
2
1459
3
1459
4
1459
5
1459
6
1459
7
1459
8
1459
9
1459
10
1459
11
1459
12
1459
13
1459
14
1460
0
1460
1
1460
2
1460
3
1460
4
1460
5
1460
6
1460
7
1460
8
1460
9
1460
10
1460
11
1460
12
1460
13
1460
14
1461
0
1461
1
1461
2
1461
3
1461
4
1461
5
1461
6
1461
7
1461
8
1461
9
1461
10
1461
11

In [3]:
V

array([[[-2.11761502, -2.07728532, -2.02365717, ..., -1.50361484,
         -1.46419955, -1.43553174],
        [-2.11742456, -2.07710198, -2.02348311, ..., -1.50351874,
         -1.46410845, -1.43544421],
        [-2.11723415, -2.07691866, -2.02330908, ..., -1.50342266,
         -1.46401737, -1.43535669],
        ...,
        [-1.86633706, -1.83486448, -1.79284879, ..., -1.37270377,
         -1.33998303, -1.31630887],
        [-1.86618914, -1.83472202, -1.79271446, ..., -1.37262948,
         -1.33991305, -1.31624296],
        [-1.86604125, -1.83457975, -1.79258029, ..., -1.37255527,
         -1.33984314, -1.31617711]],

       [[-1.93824056, -1.9003062 , -1.85038973, ..., -1.37249419,
         -1.3359453 , -1.30911675],
        [-1.9380523 , -1.90012519, -1.85021804, ..., -1.37239972,
         -1.33585582, -1.30903087],
        [-1.93786409, -1.8999442 , -1.85004639, ..., -1.37230527,
         -1.33576636, -1.308945  ],
        ...,
        [-1.6922013 , -1.66314369, -1.62472752, ..., -